In [45]:
import nest_asyncio
import os
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()

api_key = os.environ.get('OPENAI_API_KEY')


In [34]:
from agents import Agent, Runner, handoff

agent=Agent(
    name="Basic Agent",
    instructions= "You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)
result= await Runner.run(agent,"Hello how are you?")
result.final_output

"HELLO! I'M DOING WELL, THANK YOU! HOW ABOUT YOU?"

In [35]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it",
    model="gpt-4o-mini"
)

topic= "AI"
result=await Runner.run(joke_agent,topic)
result.final_output

"Why did the AI go broke?  \n\nBecause it couldn't find its cache!"

In [38]:
language_agent=Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in different languages.",
)
joke_result= await Runner.run(joke_agent,topic)
translated_result= await Runner.run(language_agent,f"Translate this joke the Turkish:{joke_result.final_output}")
print(f"Original Joke:\n{joke_result.final_output}\n")
print(f"Translated Joke:\n{translated_result.final_output}\n")

Original Joke:
Why did the AI go broke?

Because it lost its cache!

Translated Joke:
Here's the joke in Turkish:

Yapay zeka neden iflas etti?

Çünkü önbelleğini kaybetti!



# Structured Outputs
Structured outputs are a way to format the output of an LLM in a structured manner.

In [39]:
from pydantic import BaseModel

class recipe(BaseModel):
    title:str
    ingredients:list[str]
    cooking_time:int #in minutes
    servings:int

recipe_agent=Agent(
    name="Recipe Agent",
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job is to output that as an actual detailed recipe. The cooking time should be in minutes"),
    output_type=recipe
)
response = await Runner.run(recipe_agent,"Italian Risotto")
response.final_output

recipe(title='Classic Italian Risotto', ingredients=['2 cups Arborio rice', '4 cups chicken or vegetable broth, kept warm', '1 cup dry white wine', '1 onion, finely chopped', '2 cloves garlic, minced', '4 tablespoons unsalted butter', '1 cup grated Parmesan cheese', 'Salt and freshly ground black pepper, to taste', '2 tablespoons olive oil', 'Chopped fresh parsley, for garnish'], cooking_time=45, servings=4)

# Tool Calling
Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs.

In [43]:
from agents import Agent,function_tool

@function_tool
def get_weather(city:str)->str:
    print(f"Getting weather for: {city}")
    return "rainy"

@function_tool
def get_temperature(city:str)->str:
    print(f"Getting temperature for: {city}")
    print(f"Getting temperature for: {city}")
    return "70 degrees"

agent=Agent(
    name="Weather Agent",
    instructions="You are the local weather agent. You are given a city and you need to tell the weather and tempreture.",
    tools=[get_weather,get_temperature]
)

result = await Runner.run(agent,"What is the weather today?")
result.final_output

result = await Runner.run(agent,"What is the weather for Istanbul?")
result.final_output

Getting weather for: Istanbul
Getting temperature for: Istanbul
Getting temperature for: Istanbul


'The weather in Istanbul is currently rainy, with a temperature of 70 degrees Fahrenheit.'

In [44]:
"""Search Tool"""
from agents import WebSearchTool

news_agent=Agent(
    name="News Agent",
    instructions="You are a news reporter. Your job is to find recent news articles on the internet about World politics",
    tools=[WebSearchTool()]
)
result = await Runner.run(news_agent,"What is the news articles today?")
print(result.final_output)

Here are the latest developments in world politics as of June 25, 2025:

**NATO Summit in The Hague**

The 2025 NATO Summit is currently underway in The Hague, Netherlands, from June 24 to 25. This marks the first time the Netherlands has hosted a NATO summit. Discussions are centered on member states' commitments to increase defense spending to 5% of GDP. Notably, New Zealand is participating as the only non-Atlantic partner country. ([en.wikipedia.org](https://en.wikipedia.org/wiki/2025_The_Hague_NATO_summit?utm_source=openai))

**U.S. Strikes on Iranian Nuclear Facilities**

On June 21, 2025, the United States conducted airstrikes targeting Iran's Fordow uranium enrichment facility, the Natanz nuclear facility, and the Isfahan nuclear technology center. President Donald Trump stated that these actions "completely and totally obliterated" Iran's key nuclear enrichment facilities. However, initial assessments suggest that while above-ground structures were damaged, underground facilit

# Handoffs
A way for an agent to invoke another agent

In [48]:
    from agents import Agent,Runner
from pydantic import BaseModel

class Tutorial(BaseModel):
    outline:str
    tutorial:str

tutorial_generator=Agent(
    name="Tutorial Generator",
    handoff_description="Used for generating a tutorial based on an outline",
    instructions=("Given a programming topic and an outline, your job is to generate code snippets for each section of the outline. Format the tutorial in Markdown using a mix of text for explanation and code snippets for examples. Where it makes sense, include comments in the code snippets to further explain the code."),
    output_type=Tutorial

    )

outline_builder= Agent(
    name="Outline Builder",
    instructions=" Given a particular programming topic, your job is to help come up with a tutorial. You will do that by crafting an outline. After making outline, hand it to the tutorial generator agent",

)

handoffs=[tutorial_generator]

tutorial_response=await Runner.run(outline_builder,"Loop in python")
print(tutorial_response.final_output)

# Python Loops Tutorial Outline

## Introduction
- Brief explanation of why loops are important in programming
- Overview of types of loops in Python

## 1. For Loop
- Definition and syntax
- Basic examples
- Using `range()`
  - Examples using `range()` with different parameters
- Iterating over lists, tuples, and strings
  - Examples of each
- Nested for loops
  - Explanation and examples

## 2. While Loop
- Definition and syntax
- Basic examples
- Comparison with for loops
- Common use cases for while loops
  - Examples

## 3. Loop Control Statements
- Break
  - Explanation and examples
- Continue
  - Explanation and examples
- Pass
  - Explanation and examples

## 4. The Else Clause in Loops
- Explanation of the else clause
- Examples with for and while loops
- Real-world examples

## 5. Looping Techniques
- Looping with indices using `enumerate()`
  - Examples
- Looping over two or more sequences using `zip()`
  - Examples
- List comprehensions as an alternative to loops
  - Brief 

In [50]:
from agents import Agent,Runner,handoff,RunContextWrapper

history_tutor_agent=Agent(
    name="History Tutor Agent",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent=Agent(
    name="Math Tutor Agent",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples.",

)

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to math tutor agent")

def on_history_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to history tutor agent")


triage_agent=Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework questions. If neither agent is relevant, provide a general response.",
    handoffs=[
    handoff(history_tutor_agent,on_handoff=on_history_handoff),
    handoff(math_tutor_agent,on_handoff=on_math_handoff)
    ]
)

"""result = await Runner.run(triage_agent,"how do I add 2 and 4")
result.final_output"""

result2=await Runner.run(triage_agent,"how did ww2 start?")
result2.final_output

Handing off to math tutor agent
Handing off to history tutor agent


'World War II began on September 1, 1939, when Germany, led by Adolf Hitler, invaded Poland. This invasion prompted Britain and France to declare war on Germany two days later, on September 3, marking the official start of the war.\n\n### Context Leading to the War:\n\n1. **Treaty of Versailles (1919):** The harsh terms imposed on Germany after World War I created economic distress and national resentment. This environment facilitated the rise of Adolf Hitler and the Nazi Party, who promised to restore Germany’s power and pride.\n\n2. **Rise of Totalitarian Regimes:** In the 1930s, aggressive totalitarian regimes came to power in Germany, Italy (under Mussolini), and Japan, each pursuing expansionist policies.\n\n3. **Expansionist Policies:**\n   - **Germany:** Following the remilitarization of the Rhineland and the annexation of Austria in the Anschluss (1938), Hitler turned his attention to Czechoslovakia and later, Poland.\n   - **Italy:** Invaded Ethiopia in 1935.\n   - **Japan:** 

In [56]:
from agents import function_tool,trace

class ManagerEscalation(BaseModel):
    issue: str # the issue being escalated
    why: str # why can you not handle it? Used for training in the future

@function_tool
def create_ticket(issue: str):
    """"
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID: 12345"
    # In a real-world scenario, this would interact with a ticketing system

manager_agent = Agent(
    name="Manager",
    handoff_description="Handles escalated issues that require managerial attention",
    instructions=(
        "You handle escalated customer issues that the initial custom service agent could not resolve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."
    ),
    tools=[create_ticket],
)

def on_manager_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
    print("Escalating to manager agent: ", input.issue)
    print("Reason for escalation: ", input.why)

    # here we might store the escalation in a database or log it for future reference

customer_service_agent = Agent(
    name="Customer Service",
    instructions="You assist customers with general inquiries and basic troubleshooting. " +
                 "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue yourself.",
    handoffs=[handoff(
        agent=manager_agent,
        input_type=ManagerEscalation,
        on_handoff=on_manager_handoff,
    )]
)

result = await Runner.run(customer_service_agent, "I want a refund, but your system wont let me process it. The website is just blank for me.")
print(result.final_output)

"""         TRACE


with trace("Customer Service Hotline"):
    result = await Runner.run(customer_service_agent, "Hello how much are tickets?.")
    print(result.final_output)
"""

Escalating to manager agent:  Customer unable to process refund due to website displaying a blank page.
Reason for escalation:  Technical issue with the website interface preventing refund processing. Needs investigation from the technical team to resolve the system error.
Creating ticket for issue: Customer unable to process refund due to website displaying a blank page. Needs technical team to investigate and resolve the system error.
I've created a ticket for your issue, and our technical team will investigate the problem. Your ticket ID is **12345**. We appreciate your patience while we work to resolve this. If you have further questions, please let me know!


# Tracing
A way to see what the agents are doing

In [ ]:
from agents import trace

joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it",
    model="gpt-4o-mini"
)
language_agent=Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in different languages.",
)
with trace("Joke Translation Workflow"):
    joke_result=await Runner.run(joke_agent,"Cheese")
    translated_result=await Runner.run(language_agent,f"Translate this joke the Turkish:{joke_result.final_output}")

    print(f"Translated Joke:\n{translated_result.final_output}\n")